In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

train_data = pd.read_csv('adult.data', header= None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'], encoding = 'cp1252')
test_data = pd.read_csv('adult.test', header= None, skiprows=1,  names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'], encoding = 'cp1252')

od = test_data
train_data = train_data.drop(['fnlwgt'], axis=1)
test_data = test_data.drop(['fnlwgt'], axis=1)

train_data.drop(['education'], axis = 1, inplace = True)
test_data.drop(['education'], axis = 1, inplace = True)

# 每個元素的前後空格去掉
train_data = train_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
test_data = test_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#查看各欄"?"各數
train_data.apply(lambda x: np.sum(x == "?"))
test_data.apply(lambda x: np.sum(x == "?"))

#把"?"取代為NaT
train_data.replace("?", pd.NaT, inplace = True)
test_data.replace("?", pd.NaT, inplace = True)


#將缺失值補齊，名目資料填眾數，數值資料填平均值。
fill_data = {'workclass': train_data['workclass'].mode()[0], 'occupation': train_data['occupation'].mode([0]), 'native-country': train_data['native-country'].mode()[0]}
fill_data_test = {'workclass': test_data['workclass'].mode()[0], 'occupation': test_data['occupation'].mode([0]), 'native-country': test_data['native-country'].mode()[0]}

train_data.fillna(fill_data, inplace=True)
test_data.fillna(fill_data_test, inplace=True)

#將income欄位>50K改為1，<=50K改為0
train_data['income'] = train_data['income'].apply(lambda x: 0 if x == "<=50K" else 1)
test_data['income'] = test_data['income'].apply(lambda x: 0 if x == '<=50K.' else 1)

train_data = pd.get_dummies(train_data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], dtype=int)

test_data = pd.get_dummies(test_data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], dtype=int)
test_data['native-country_Holand-Netherlands'] = 0


feature_mapping = {
    'native-country_Trinadad&Tobago': 'native-country_Trinadad_and_Tobago',
    'native-country_Outlying-US(Guam-USVI-etc)': 'native-country_Outlying-US-Guam-USVI-etc'
}

train_data.rename(columns=feature_mapping, inplace=True)
test_data.rename(columns=feature_mapping, inplace=True)

train_data = train_data.sort_index(axis=1)
test_data = test_data.sort_index(axis=1)
train_data
# columns = train_data.columns
# with open("columns.csv", "w") as f:
#     f.write("\n".join(columns))

,age,capital-gain,capital-loss,education-num,hours-per-week,income,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,...,sex_Female,sex_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,39,2174,0,13,40,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,50,0,0,13,13,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,38,0,0,9,40,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,53,0,0,7,40,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
4,28,0,0,13,40,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,0,0,12,38,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
32557,40,0,0,9,40,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
32558,58,0,0,9,40,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
32559,22,0,0,9,20,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [44]:
# Extract numerical columns
numerical_columns = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the scaler on the training data
train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data[numerical_columns]), columns=numerical_columns)
train_data[numerical_columns] = train_data_scaled

# Transform the test data using the same scaler
test_data_scaled = pd.DataFrame(scaler.transform(test_data[numerical_columns]), columns=numerical_columns)
test_data[numerical_columns] = test_data_scaled
train_data

,age,capital-gain,capital-loss,education-num,hours-per-week,income,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,...,sex_Female,sex_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,0.301370,0.021740,0.0,0.800000,0.397959,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0.452055,0.000000,0.0,0.800000,0.122449,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,0.287671,0.000000,0.0,0.533333,0.397959,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0.493151,0.000000,0.0,0.400000,0.397959,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
4,0.150685,0.000000,0.0,0.800000,0.397959,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.136986,0.000000,0.0,0.733333,0.377551,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
32557,0.315068,0.000000,0.0,0.533333,0.397959,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
32558,0.561644,0.000000,0.0,0.533333,0.397959,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
32559,0.068493,0.000000,0.0,0.533333,0.193878,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [45]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

from rpy2.robjects.packages import importr

utils = robjects.packages.importr('utils')
C50 = robjects.packages.importr("C50")
partykit = robjects.packages.importr("partykit")



# graph = robjects.packages.importr("graph")


robjects.r('Sys.setlocale("LC_ALL", "en_US.UTF-8")')

robjects.r('library(C50)')

pandas2ri.activate()

r_train_data = pandas2ri.py2rpy(train_data)
r_test_data = pandas2ri.py2rpy(test_data)


# with (robjects.default_converter + pandas2ri.converter).context():
#   r_train_data = robjects.conversion.get_conversion().py2rpy(train_data)

# with (robjects.default_converter + pandas2ri.converter).context():
#   r_test_data = robjects.conversion.get_conversion().py2rpy(test_data)



# r_train_data = pandas2ri.conversion.py2rpy(train_data)
# r_test_data = pandas2ri.conversion.py2rpy(test_data)

robjects.r.assign("r_train_data", r_train_data)
robjects.r.assign("r_test_data", r_test_data)

# 在 R 中建立 C4.5 決策樹模型
# robjects.r('print(r_test_data$"income")')
robjects.r('col<-r_test_data$"income"')




In [46]:
# robjects.r('formula <- income ~ age + capital-gain + capital-loss + education-num + hours-per-week + marital-status_Divorced + marital-status_Married-AF-spouse + marital-status_Married-civ-spouse + marital-status_Married-spouse-absent + marital-status_Never-married + marital-status_Separated + marital-status_Widowed + native-country_Cambodia + native-country_Canada + native-country_China + native-country_Columbia + native-country_Cuba + native-country_Dominican-Republic + native-country_Ecuador + native-country_El-Salvador + native-country_England + native-country_France + native-country_Germany + native-country_Greece + native-country_Guatemala + native-country_Haiti + native-country_Holand-Netherlands + native-country_Honduras + native-country_Hong + native-country_Hungary + native-country_India + native-country_Iran + native-country_Ireland + native-country_Italy + native-country_Jamaica + native-country_Japan + native-country_Laos + native-country_Mexico + native-country_Nicaragua + native-country_Outlying-US(Guam-USVI-etc) + native-country_Peru + native-country_Philippines + native-country_Poland + native-country_Portugal + native-country_Puerto-Rico + native-country_Scotland + native-country_South + native-country_Taiwan + native-country_Thailand + native-country_Trinadad&Tobago + native-country_United-States + native-country_Vietnam + native-country_Yugoslavia + occupation_Adm-clerical + occupation_Armed-Forces + occupation_Craft-repair + occupation_Exec-managerial + occupation_Farming-fishing + occupation_Handlers-cleaners + occupation_Machine-op-inspct + occupation_Other-service + occupation_Priv-house-serv + occupation_Prof-specialty + occupation_Protective-serv + occupation_Sales + occupation_Tech-support + occupation_Transport-moving + race_Amer-Indian-Eskimo + race_Asian-Pac-Islander + race_Black + race_Other + race_White + relationship_Husband + relationship_Not-in-family + relationship_Other-relative + relationship_Own-child + relationship_Unmarried + relationship_Wife + sex_Female + sex_Male + workclass_Federal-gov + workclass_Local-gov + workclass_Never-worked + workclass_Private + workclass_Self-emp-inc + workclass_Self-emp-not-inc + workclass_State-gov + workclass_Without-pay')

robjects.r('colnames(r_train_data)[colnames(r_train_data) == "income"] <- "class"')
robjects.r('r_train_data$class <- as.factor(r_train_data$class)')
robjects.r('subset_data <- r_train_data[, !colnames(r_train_data) %in% "class"]')

robjects.r('model <- C5.0(x = subset_data, y = r_train_data$class, trials = 8, control = C5.0Control(CF = FALSE, label = "class"))')

model = robjects.globalenv['model']



# robjects.r('print(summary(model))')

In [47]:
# robjects.r('colnames(r_test_data)[colnames(r_test_data) == "income"] <- "class"')
robjects.r('subset_data_test <- r_test_data[, !colnames(r_test_data) %in% "income"]')
r_predictions = robjects.r('''
    # 將模型套用在測試數據                 
    predictions <- predict(model, newdata = subset_data_test)         
    freq1 <-table(predictions, col)
                           
    accuracy<-sum(diag(freq1)/sum(freq1))
    print(accuracy)
''')

[1] 0.8673914


In [48]:
#將 R 的預測結果轉換為 Python 陣列
#rpy2

predictions = robjects.globalenv['predictions']

with (robjects.default_converter + pandas2ri.converter).context():
  pd_from_r_df = robjects.conversion.get_conversion().rpy2py(predictions)

from openpyxl import Workbook
#產出Excel(Test data)
wb = Workbook()
ws = wb.active
ws.append(['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income','Predict result'])

for i in range(len(pd_from_r_df)):
    if pd_from_r_df[i] == '0':
        result = '<=50K.'
    else:
        result = '>50K.'
    #將現在loop到原始資料的列轉為list
    li = od.iloc[i,:].tolist()
    #
    li.append(result)
    ws.append(li)
wb.save('coolData.xlsx')
# predictions = np.array(predictions)

print(pd_from_r_df)

['0', '0', '0', '1', '0', ..., '0', '0', '1', '0', '1']
Length: 16281
Categories (2, object): ['0', '1']
